# Exercise 07

## Kaggle competition 

* Overview of how Kaggle works ([slides](https://github.com/justmarkham/DAT8/raw/master/slides/16_kaggle.pdf))
* Kaggle Titanic competition: [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

# Exercise 07.1 (20 points)

Create a submission using different classification methods, preprocessing extrategies and cross-validation techniques discussed during the class. The output must be detailed in this notebook.

# Exercise 07.2 (20 points)

The reminder points will be allocated based on the performance of each one submission.

# Titanic

Kaggle Competition es una plataforma para el modelamiento y análisis predictivvo de competencias en la que empresas publican datos y personas de todo el mundo compiten para producir los mejores modelos, existen un sin número de estrategias que se pueden aplicar al modelamiento y es difícil saber en principio  cual es la técnica más eficaz.

A través de la plataforma se obtiene la base de datos llamada Titanic en la cual se tiene información de los pasajeros que abordaron el transatlántico y lo que se quiere es predecir que pasajeros sobreviven a la tragedia a demás conocer que personas eran propensas a sobrevivir; de antemano se conoce que algunos grupos de personas tenían más probabilidades de sobrevivir que otros, como ser mujer, ser niño y pertencer a la clase social alta.

Las variables que se tienen son las siguientes:

* Survived - Supervivencia (0 = No, 1 = Sí)

* PClass - Clase de los Pasajeros (1 = 1º Clase, 2 = 2º Clase, 3 = 3º Clase)

* Name - Nombre

* Sex - Sexo

* Age - Edad

* Sibsp - Número de hermanos / cónyuges a bordo

* Parch - Número de padres / Niños a bordo

* Ticket - N° de entrada

* Fare - Tarifas de los pasajeros

* Cabin - Cabina

* Embarked - Puerto de embarque (C = Cherburgo; Q = Queenstown; S = Southampton)

# 1. Primer Modelamiento 

Para comenzar a realizar los diferentes modelos primeramente se divide la base train_df en train y test, para poder entrenar el modelo en train y aplicarlo a test y conocer de antemano la exactitud del modelo, continuamente se aplica el modelo a test_df y se suben los resultados a Kaggle que muestra cual es la precisión del valor obtenido vs el valor real.

Después de dividir la base se reemplazan los valores perdidos en train_df que es renombra como titanic para Age y Embarked con la media y la moda respectivamente a demás se convierte la variable Embarked en variables dummy y la variable Sex en variable binaria. Continuamente se crean nuevas variables en donde se elevan al cuadrado Age, SibSp, Parch y Fare como una estrategia para mejorar las predicciones en los modelos (estrategia que se pondrá a prueba) ya que si se tiene una relación cuadrática entre estas variables y la supervivencia convendría elevarlas al cuadrado.

Al aplicar el modelo y conocer su accuracy se realiza lo mismo que se realizó para titanic en test_df y es reemplazar los valores perdidos (con los valores de titanic), realizar las variables dummy, la variable binaria y crear las mismas variables elevadas al cuadrado para poder aplicar el modelo.

Con las variables descritas anteriormente se van hacer los siguientes modelos para obtener la predicción:

* 1.1. Logistic Regression
* 1.2. Cross Validation
* 1.3. Multinomial Naive Bayes
* 1.4. Gaussian Naive Bayes

# 1.1. Logistic Regression

In [4]:
import pandas as pd
import numpy as np

In [5]:
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [6]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)

In [7]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Sex_Female       0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
dtype: int64

In [8]:
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [84]:
titanic.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_C,Embarked_Q,Embarked_S,Age2,SibSp2,Parch2,Fare2
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,0,0,0,1,484,1,0,52.562500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,1,0,0,1444,1,0,5081.308859


In [9]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_Q', 'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.80269058296


In [86]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [87]:
test_df.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Female,Embarked_C,Embarked_Q,Embarked_S,Age2,SibSp2,Parch2,Fare2
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0,1,0,1190.25,0,0,61.296373
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0,0,1,2209.00,1,0,49.000000


In [88]:
predictions = logreg.predict(test_df[feature_cols])

In [89]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission1.csv", index=False)

# 1.2. Cross Validation

In [10]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [11]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [12]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C','Embarked_Q', 
                'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# Create k-folds
from sklearn.cross_validation import KFold
kf = KFold(X.shape[0], n_folds=10, random_state=0)

results = []
models = []
for train_index, test_index in kf:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # train a logistic regression model
    models.append(LogisticRegression(C=1e9))
    
    models[-1].fit(X_train, y_train)

    # make predictions for testing set
    y_pred_class = models[-1].predict(X_test)

    # calculate testing accuracy
    results.append(metrics.accuracy_score(y_test, y_pred_class))
results

[0.78888888888888886,
 0.8314606741573034,
 0.7640449438202247,
 0.8539325842696629,
 0.7865168539325843,
 0.7865168539325843,
 0.7865168539325843,
 0.7640449438202247,
 0.8651685393258427,
 0.8314606741573034]

In [93]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [94]:
probas = pd.DataFrame(index=test_df[feature_cols].index, columns=['p'+ str(i) for i in range(10)])

In [95]:
for i in range(10):
    proba = models[i].predict_proba(test_df[feature_cols])[:, 1]
    probas.iloc[:, i] = proba

In [96]:
predictions = ((probas.mean(axis=1) > 0.5) * 1.0).values
predictions = predictions.astype(np.int)

In [28]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_5.csv", index=False)

# 1.3. Multinomial Naive Bayes

In [97]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [98]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [99]:
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_Q', 'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [100]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.61434977578475336

In [101]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [102]:
predictions = mnb.predict(test_df[feature_cols])

In [35]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission8.csv", index=False)

# 1.4. Gaussian Naive Bayes

In [104]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [105]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [106]:
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_Q', 'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [107]:
# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.72197309417040356

In [108]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [109]:
predictions = gnb.predict(test_df[feature_cols])

In [33]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission9.csv", index=False)

Al realizar los anteriores modelos y subirlos a Kaggle se puede determinar que el modelo que tuvo el mejor modelo fue la regresión logística y el modelo que tuvo la peor predicción fue MultinomialNB. Lo que se realizará a continución será eliminar variables que posiblemente este aportando muy poco al modelo se utilizan tres métodos:

* 2.1. Eliminación de variables con poca varianza: Se eliminan todos los elementos cuya varianza no cumple con algún umbral. Por defecto, se eliminan todas las variables con varianza cero, es decir, características que tienen el mismo valor en todas las muestras.

* 2.2. Selección de variables univariadas: SelectPercentile elimina pero un porcentaje más alto de puntuación variables especificado por el usuario mediante pruebas estadísticas univariantes comunes para cada característica: tasa de falsos positivos SelectFpr, tasa de falso descubrimiento SelectFdr, o la familia de error SelectFwe.

* 2.3. Función recursiva de eliminación: Tiene en cuenta un estimador externo que asigna ponderaciones a las variables, el estimador es entrenado en el conjunto inicial de variables y los pesos son asignados a cada uno de ellos. Entonces, cuenta los pesos absolutos más pequeños y estos se podan del conjunto de variables actual, hasta que finalmente se alcanza el número deseado de características para seleccionar.

Después de conocer cuales son las variables que no aportan al modelo según cada uno de los tres métodos se procede a correr nuevamente los cuatro métodos realizados en el Primer Punto sin dichas variables.

# 2. Eliminación de Variables

# 2.1. Eliminación de Variables con poca varianza

In [34]:
features = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C','Embarked_Q', 'Embarked_S',
            'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[list(features)]
y = titanic['Survived']

In [35]:
results = pd.DataFrame(index=['k='+ str(i) for i in range(1, 14)], columns=['mean_accuracy'])

In [36]:
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
t=[]

for i in range(1,14):
    sel = SelectKBest(k=i)
    sel.fit(X, y)
    sel.get_support()
    X_sel = sel.transform(X)
    results.loc['k='+str(i)] = pd.Series(cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy')).mean()

results

,mean_accuracy
k=1,0.786698
k=2,0.786698
k=3,0.782229
k=4,0.775525
k=5,0.774426
k=6,0.771043
k=7,0.792354
k=8,0.790132
k=9,0.79575
k=10,0.802517


In [37]:
results.idxmax()

mean_accuracy    k=12
dtype: object

In [38]:
from sklearn.feature_selection import SelectKBest

sel = SelectKBest(k=12)
sel.fit(X, y)
sel.get_support()
print(np.array(features)[~sel.get_support()])

['Embarked_Q']


# 2.2. Selección de variables univariadas

In [39]:
features = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C','Embarked_Q', 'Embarked_S', 
            'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[list(features)]
y = titanic['Survived']

In [40]:
from sklearn.feature_selection import SelectPercentile, f_classif

results = pd.DataFrame(index=range(99), columns=['mean_accuracy'])

for i in range(1,100):
    sel = SelectPercentile(f_classif, percentile=i)
    sel.fit(X, y)
    sel.get_support()
    X_sel = sel.transform(X)
    results.iloc[i-1] = pd.Series(cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy')).mean()

In [41]:
results.idxmax()

mean_accuracy    91
dtype: int64

In [42]:
results.iloc[91]

mean_accuracy    0.803654
Name: 91, dtype: object

In [43]:
from sklearn.feature_selection import SelectPercentile, f_classif

sel = SelectPercentile(f_classif, percentile=91)
sel.fit(X, y)
sel.get_support()
print(np.array(features)[~sel.get_support()])

['Embarked_Q' 'Parch2']


# 2.3. Función recursiva de eliminación

In [44]:
features = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C','Embarked_Q', 'Embarked_S', 
            'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[list(features)]
y = titanic['Survived']

In [45]:
from sklearn.feature_selection import RFE

results = pd.DataFrame(index=range(13), columns=['mean_accuracy'])

for i in range(1,14):
    sel = RFE(estimator=logreg, n_features_to_select=i)
    sel.fit(X, y)
    X_sel = sel.transform(X)
    results.iloc[i-1] = cross_val_score(logreg, X_sel, y, cv=10, scoring='accuracy').mean()
results

,mean_accuracy
0,0.786698
1,0.786698
2,0.772179
3,0.771055
4,0.784526
5,0.782304
6,0.79017
7,0.796886
8,0.796886
9,0.802543


In [46]:
results.idxmax()

mean_accuracy    11
dtype: int64

In [47]:
from sklearn.feature_selection import RFE
sel = RFE(estimator=logreg, n_features_to_select=11)
sel.fit(X, y)
print(np.array(features)[~sel.get_support()])

['Age2' 'Fare2']


Por medio de los tres métodos de eliminación se conocen cuales son las variables que se van a utilizar para volver a realizar el modelamiento de los datos y concluir si mejoran las precisiones de los modelos.

# 3. Primer Modelamiento Eliminando Variables

# 3.1. Logistic Regression Eliminando 'Embarked_Q'

In [110]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [111]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [112]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 'Embarked_S', 'Age2',
                'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.807174887892


In [113]:
test_df.Age.fillna(test_df.Age.mean(), inplace=True) 
test_df.Fare.fillna(test_df.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [114]:
predictions = logreg.predict(test_df[feature_cols])

In [53]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission2.csv", index=False)

# 3.2. Logistic Regression Eliminando 'Embarked_Q' 'Parch2'

In [115]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [116]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [117]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 'Embarked_S', 'Age2', 'SibSp2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.798206278027


In [118]:
test_df.Age.fillna(test_df.Age.mean(), inplace=True) 
test_df.Fare.fillna(test_df.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [119]:
predictions = logreg.predict(test_df[feature_cols])

In [59]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission3.csv", index=False)

# 3.3. Logistic Regression Eliminando 'Age2' 'Fare2'

In [60]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [61]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [62]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C','Embarked_Q', 'Embarked_S', 'SibSp2','Parch2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.793721973094


In [63]:
test_df.Age.fillna(test_df.Age.mean(), inplace=True) 
test_df.Fare.fillna(test_df.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [64]:
predictions = logreg.predict(test_df[feature_cols])

In [65]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission4.csv", index=False)

# 3.4. Cross Validation Elimando 'Embarked_Q'

In [69]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [70]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [105]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 
                'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# Create k-folds
from sklearn.cross_validation import KFold
kf = KFold(X.shape[0], n_folds=10, random_state=0)

results = []
models = []
for train_index, test_index in kf:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # train a logistic regression model
    models.append(LogisticRegression(C=1e9))
    
    models[-1].fit(X_train, y_train)

    # make predictions for testing set
    y_pred_class = models[-1].predict(X_test)

    # calculate testing accuracy
    results.append(metrics.accuracy_score(y_test, y_pred_class))
results

[0.78888888888888886,
 0.8202247191011236,
 0.7640449438202247,
 0.84269662921348309,
 0.7865168539325843,
 0.7865168539325843,
 0.7865168539325843,
 0.7640449438202247,
 0.8539325842696629,
 0.8089887640449438]

In [68]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [107]:
probas = pd.DataFrame(index=test_df[feature_cols].index, columns=['p'+ str(i) for i in range(10)])

In [108]:
for i in range(10):
    proba = models[i].predict_proba(test_df[feature_cols])[:, 1]
    probas.iloc[:, i] = proba

In [109]:
predictions = ((probas.mean(axis=1) > 0.5) * 1.0).values
predictions = predictions.astype(np.int)

In [110]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_51.csv", index=False)

# 3.5. Cross Validation Eliminando 'Embarked_Q' 'Parch2'

In [13]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [14]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [15]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 
                'Embarked_S', 'Age2', 'SibSp2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# Create k-folds
from sklearn.cross_validation import KFold
kf = KFold(X.shape[0], n_folds=10, random_state=0)

results = []
models = []
for train_index, test_index in kf:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # train a logistic regression model
    models.append(LogisticRegression(C=1e9))
    
    models[-1].fit(X_train, y_train)

    # make predictions for testing set
    y_pred_class = models[-1].predict(X_test)

    # calculate testing accuracy
    results.append(metrics.accuracy_score(y_test, y_pred_class))
results

[0.76666666666666672,
 0.8314606741573034,
 0.7865168539325843,
 0.8539325842696629,
 0.7752808988764045,
 0.7865168539325843,
 0.7865168539325843,
 0.7752808988764045,
 0.8651685393258427,
 0.8089887640449438]

In [16]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [17]:
probas = pd.DataFrame(index=test_df[feature_cols].index, columns=['p'+ str(i) for i in range(10)])

In [18]:
for i in range(10):
    proba = models[i].predict_proba(test_df[feature_cols])[:, 1]
    probas.iloc[:, i] = proba

In [19]:
predictions = ((probas.mean(axis=1) > 0.5) * 1.0).values
predictions = predictions.astype(np.int)

In [20]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_52.csv", index=False)

# 3.6. Cross Validation Eliminando 'Age2' 'Fare2'

In [127]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [128]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [129]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C','Embarked_Q', 
                'Embarked_S', 'SibSp2','Parch2']
X = titanic[feature_cols]
y = titanic.Survived

# Create k-folds
from sklearn.cross_validation import KFold
kf = KFold(X.shape[0], n_folds=10, random_state=0)

results = []
models = []
for train_index, test_index in kf:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # train a logistic regression model
    models.append(LogisticRegression(C=1e9))
    
    models[-1].fit(X_train, y_train)

    # make predictions for testing set
    y_pred_class = models[-1].predict(X_test)

    # calculate testing accuracy
    results.append(metrics.accuracy_score(y_test, y_pred_class))
results

[0.76666666666666672,
 0.8314606741573034,
 0.7640449438202247,
 0.8539325842696629,
 0.7752808988764045,
 0.7865168539325843,
 0.7865168539325843,
 0.7752808988764045,
 0.8539325842696629,
 0.84269662921348309]

In [130]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [131]:
probas = pd.DataFrame(index=test_df[feature_cols].index, columns=['p'+ str(i) for i in range(10)])

In [132]:
for i in range(10):
    proba = models[i].predict_proba(test_df[feature_cols])[:, 1]
    probas.iloc[:, i] = proba

In [133]:
predictions = ((probas.mean(axis=1) > 0.5) * 1.0).values
predictions = predictions.astype(np.int)

In [134]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_53.csv", index=False)

# 3.7. Multinomial NB Eliminando 'Embarked_Q'

In [74]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [75]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [76]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [77]:
# import both Multinomial and Gaussian Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics
# testing accuracy of Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.61434977578475336

In [78]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [ ]:
predictions = mnb.predict(test_df[feature_cols])
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission81.csv", index=False)

# 3.8. Multinomial NB Eliminando 'Embarked_Q' 'Parch2'

In [84]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [85]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [86]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 'Embarked_S', 'Age2', 'SibSp2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [87]:
# import both Multinomial and Gaussian Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics
# testing accuracy of Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.61434977578475336

In [88]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [89]:
predictions = mnb.predict(test_df[feature_cols])
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission82.csv", index=False)

# 3.9. Multinomial NB Eliminando 'Age2' 'Fare2'

In [90]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [91]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [92]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_Q', 'Embarked_S', 'SibSp2','Parch2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [93]:
# import both Multinomial and Gaussian Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics

In [94]:
# testing accuracy of Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.68161434977578472

In [95]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [96]:
predictions = mnb.predict(test_df[feature_cols])

In [97]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission83.csv", index=False)

# 3.10. Gaussian NB Eliminando 'Embarked_Q'

In [43]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [44]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [45]:
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_Q', 'Embarked_S', 'Age2', 'SibSp2','Parch2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [46]:
# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.72197309417040356

In [47]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [48]:
predictions = gnb.predict(test_df[feature_cols])

In [ ]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission91.csv", index=False)

# 3.11. Gaussian NB Eliminando 'Embarked_Q' 'Parch2'

In [49]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [50]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [51]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 'Embarked_S', 'Age2', 'SibSp2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [52]:
# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.69506726457399104

In [53]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [54]:
predictions = gnb.predict(test_df[feature_cols])

In [ ]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission92.csv", index=False)

# 3.12. Gaussian NB Eliminando 'Age2' 'Fare2'

In [72]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [73]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [74]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C',
                'Embarked_Q', 'Embarked_S', 'SibSp2','Parch2']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [75]:
# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
metrics.accuracy_score(y_test, y_pred_class)

0.726457399103139

In [76]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2

In [77]:
predictions = gnb.predict(test_df[feature_cols])

In [78]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission93.csv", index=False)

Al probar los cuatro modelos con las tres eliminaciones de variables que se obtuvieron en el paso 2. se concluye que el mejor modelo se obtiene por medio de la validación cruzada elimando las variables 'Embarked_Q' 'Parch2', y el de MultinomialNB para estos datos realmente tiene malos pronósticos así se realice la eliminación de variables que aportan poco al modelo y en general para todos los modelos eliminar 'Age2' 'Fare2'disminuye en gran medida el accuracy por lo que se considera que estas dos variables influyen de manera positiva en el modelo.

En la validación cruzada se observó que se mejoró la precisión del modelo al eliminar 'Embarked_Q' lo cual puede ser dado porque el haber subido en este puerto no tuvo influencia sobre la sobrevivencia, posiblemente en este puerto abordaron todo tipo de personas en cuanto a sexo, edad o estrato social. De igual manera mejora el modelo al eliminar 'Parch2' puesto que la variable con el número de padres o hijos a bordo no tendría sentido elevarla al cuadrado porque esta variable no tiene una función cuadrática con la supervivencia puesto que entre en general si la personas tenía hijos (1 o varios) era más factible que se salvaran y esto está asociado a otros factores como ser mujer o ser de clase alta.

Por lo que se puede concluir que variables como la edad, el sexo y la clase son muy importantes ya que era más probable que se salvaran las mujeres, los niños y personas de clase alta. Se pueden obtener mejores resultados si se realiza un modelo para imputar los valores perdidos en la edad ya que son bastantes o realizar otro tipo de modelo con mayor poder de predición.